In [5]:
import numpy as np
import cupy as cp
import cupyx.scipy.sparse as sp
from cupyx.scipy.sparse.linalg import gmres
from scipy.sparse.linalg import cg
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as scp
np.set_printoptions(linewidth=300, precision=3)

from qmat import QDELTA_GENERATORS

from pySDC.core.collocation import CollBase
from pySDC.implementations.problem_classes.HeatEquation_ND_FD import heatNd_unforced
from pySDC.implementations.problem_classes.AllenCahn_2D_FD import allencahn_fullyimplicit
from pySDC.implementations.problem_classes.GeneralizedFisher_1D_FD_implicit import generalized_fisher
from pySDC.implementations.problem_classes.Auzinger_implicit import auzinger

class counter(object):
    def __init__(self, disp=False):
        self._disp = disp
        self.niter = 0
    def __call__(self, rk=None):
        self.niter += 1
        if self._disp:
            # print('iter %3i\trk = %s' % (self.niter, str(rk)))
            print('   LIN: %3i' % self.niter)
    def get_iter(self):
        return self.niter

In [21]:
foo = cp.arange(0, 8)
print(foo)
y1 = foo[::2]
y2 = foo[1::2]
diag = cp.empty(len(foo))
upper = cp.zeros(len(foo)-1)
lower = cp.zeros(len(foo)-1)

diag[::2] = 1-3 * cp.power(y1, 2) - cp.power(y2, 2)
diag[1::2] = 3-3 * cp.power(y1, 2) - 9*cp.power(y2, 2)
upper[::2] = y2
lower[::2] = y1
sp.diags([upper, diag, lower], offsets=[1, 0, -1]).toarray()

[0 1 2 3 4 5 6 7]


array([[   0.,    1.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,   -6.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,  -20.,    3.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    2.,  -90.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,  -72.,    5.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    4., -270.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0., -156.,    7.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    6., -546.]])

In [68]:
def nonlinear_sdc_auzinger(dt = 0.0005, num_nodes = 4, nvars = (64, 64), nu = 2, eps=0.04,
                  abs_tol = 1E-12, rel_tol = 1E-12, k_max = 10_000, n_max = 10_000):
    # instantiate problem
    prob = auzinger()

    coll = CollBase(num_nodes=num_nodes, tleft=0, tright=1, node_type='LEGENDRE', quad_type='RADAU-RIGHT')

    generator = QDELTA_GENERATORS['MIN-SR-S'](
        # for algebraic types (LU, ...)
        Q=coll.generator.Q,
        # for MIN in tables, MIN-SR-S ...
        nNodes=coll.num_nodes,
        nodeType=coll.node_type,
        quadType=coll.quad_type,
        # for time-stepping types, MIN-SR-NS
        nodes=coll.nodes,
        tLeft=coll.tleft,
    )
    QDmat = cp.asarray(generator.genCoeffs(k=None))


    # shrink collocation matrix: first line and column deals with initial value, not needed here
    Q = cp.asarray(coll.Qmat[1:, 1:])

    u0 = cp.asarray(prob.u_exact(t=0).flatten())
    # fill in u0-vector as right-hand side for the collocation problem
    u0_coll = cp.kron(cp.ones(coll.num_nodes), u0)

    nvars = 2

    uk = cp.zeros(nvars * coll.num_nodes, dtype='float64')
    fk = cp.zeros(nvars * coll.num_nodes, dtype='float64')
    rhs = cp.zeros(nvars * coll.num_nodes, dtype='float64')

    ksum = 0
    nsum = 0
    timepoints = cp.arange(0, 0.1+dt, step=dt)
    u_buffer = cp.empty((len(timepoints), nvars))
    conv_buffer = cp.zeros(len(timepoints)) # marks the iteration, reaching the disered tolerances
    conv_buffer[0] = 1
    u_buffer[0, :] = u0

    count = counter()
    for index, timepoint in enumerate(timepoints[:-1]):
        k = 0
        #print(timepoint)
        while k < k_max:
            # eval rhs
            for m in range(coll.num_nodes):
                fk[m * nvars: (m + 1) * nvars] = cp.asarray(prob.eval_f(uk[m * nvars: (m + 1) * nvars].get(),
                                                             t=timepoint + dt * coll.nodes[m]).flatten())
    
            resnorm = cp.linalg.norm(u0_coll - (uk - dt * sp.kron(Q, sp.eye(nvars)).dot(fk)), cp.inf)
            #print('SDC:', k, ksum, resnorm)
            if resnorm < abs_tol and resnorm/cp.linalg.norm(u0_coll, cp.inf) < rel_tol:
                conv_buffer[index+1] = 1 # this iteration converged
                break
    
            g = cp.zeros(nvars * coll.num_nodes, dtype='float64')
            vn = uk
            fn = cp.zeros(nvars * coll.num_nodes, dtype='float64')
            n = 0
            while n < n_max:
                for m in range(coll.num_nodes):
                    fn[m * nvars: (m + 1) * nvars] = cp.asarray(prob.eval_f(vn[m * nvars: (m + 1) * nvars].get(),
                                                                 t=timepoint + dt * coll.nodes[m]).flatten())
                g[:] = u0_coll + dt * sp.kron(Q-QDmat, sp.eye(nvars)).dot(fk) - (vn - dt * sp.kron(QDmat, sp.eye(nvars)).dot(fn))
    
                # if g is close to 0, then we are done
                res_newton = cp.linalg.norm(g, cp.inf)
                rel_err = cp.linalg.norm(vn-uk, cp.inf) 
                #print('  Newton:', n, res_newton)
                n += 1
                nsum += 1
                if res_newton < abs_tol and rel_err < rel_tol * cp.linalg.norm(vn, cp.inf):
                    break
    
                # assemble dg
                y1 = vn[::2]
                y2 = vn[1::2]
                diag = cp.empty(len(vn))
                upper = cp.zeros(len(vn)-1)
                lower = cp.zeros(len(vn)-1)
                
                diag[::2] = 1-3 * cp.power(y1, 2) - cp.power(y2, 2)
                diag[1::2] = 3-3 * cp.power(y1, 2) - 9*cp.power(y2, 2)
                upper[::2] = -2*y2-1
                lower[::2] = 2*y1+1
                dg = -sp.eye(nvars * coll.num_nodes) + dt * sp.kron(QDmat, sp.eye(nvars)) @ sp.diags([upper, diag, lower], offsets=[1, 0, -1])
    
                # Newton
                vn -= sp.linalg.spsolve(dg, g)
                #vn -= gmres(dg, g, x0=cp.zeros_like(vn), maxiter=1000, atol=abs_tol, callback=count, callback_type='pr_norm')[0]
    
            uk = vn.copy()
    
            k += 1
            ksum += 1
        temp = uk[-nvars:]
        u_buffer[index+1, :] = temp
        u0_coll = cp.kron(cp.ones(coll.num_nodes), temp).flatten()

    return u_buffer, timepoints, prob, ksum, nsum, conv_buffer

In [70]:
%%time
u1, t1, prob, ksum, nsum, conv_buffer = nonlinear_sdc_auzinger(dt=5E-6, num_nodes = 4, abs_tol=1E-14)
print((ksum, nsum))
cp.linalg.norm(u1[-1]-cp.asarray(prob.u_exact(t1[-1]).flatten()), np.inf)

(40001, 99900)
CPU times: user 10min 20s, sys: 424 ms, total: 10min 20s
Wall time: 10min 20s


array(1.21694321e-12)

In [56]:
dts = 5*cp.logspace(-3, -6, num=7)
print(dts)
norms_auz = cp.empty(len(dts))

for index, dt in enumerate(dts):
    print("loop index: " + str(dt))
    u, t, prob, ksum, nsum, conv= nonlinear_sdc_auzinger(dt = float(dt), num_nodes = 4, abs_tol=1E-14)
    u_exact_allen =cp.asarray(prob.u_exact(t[-1]))
    norms_auz[index] = cp.max(cp.abs(u[-1] - u_exact_allen.flatten()))

[5.00000000e-03 1.58113883e-03 5.00000000e-04 1.58113883e-04
 5.00000000e-05 1.58113883e-05 5.00000000e-06]
loop index: 0.005
loop index: 0.0015811388300841897
loop index: 0.0005
loop index: 0.00015811388300841894
loop index: 4.9999999999999996e-05
loop index: 1.5811388300841898e-05
loop index: 4.9999999999999996e-06


KeyboardInterrupt: 

In [ ]:
norms_auz

# Newton-SDC

In [52]:
def nonlinear_auzinger(dt = 0.0005, num_nodes = 4, abs_tol = 1E-12, rel_tol = 1E-12, k_max = 10_000, n_max = 10_000):
    # instantiate problem
    prob = auzinger()

    coll = CollBase(num_nodes=num_nodes, tleft=0, tright=1, node_type='LEGENDRE', quad_type='RADAU-RIGHT')

    generator = QDELTA_GENERATORS['MIN-SR-S'](
        # for algebraic types (LU, ...)
        Q=coll.generator.Q,
        # for MIN in tables, MIN-SR-S ...
        nNodes=coll.num_nodes,
        nodeType=coll.node_type,
        quadType=coll.quad_type,
        # for time-stepping types, MIN-SR-NS
        nodes=coll.nodes,
        tLeft=coll.tleft,
    )
    QDmat = cp.asarray(generator.genCoeffs(k=None))

    # shrink collocation matrix: first line and column deals with initial value, not needed here
    Q = cp.asarray(coll.Qmat[1:, 1:])

    # c = coll.nodes
    # V = cp.fliplr(cp.vander(c))
    # C = cp.diag(c)
    # R = cp.diag([1 / i for i in range(1,coll.num_nodes+1)])
    # print(C @ V @ R @ cp.linalg.inv(V) - Q)
    # exit()


    u0 = cp.asarray(prob.u_exact(t=0).flatten())
    # fill in u0-vector as right-hand side for the collocation problem
    u0_coll = cp.kron(cp.ones(coll.num_nodes), u0)

    nvars =2

    un = cp.zeros(nvars * coll.num_nodes, dtype='float64')
    fn = cp.zeros(nvars * coll.num_nodes, dtype='float64')
    g = cp.zeros(nvars * coll.num_nodes, dtype='float64')
    # vk = cp.zeros(nvars * coll.num_nodes, dtype='float64')

    ksum = 0
    nsum = 0
    timepoints = cp.arange(0, cp.pi/4+dt, step=dt)
    u_buffer = cp.empty((len(timepoints), nvars))
    u_buffer[0, :] = u0
    count = counter()
    
    for index, timepoint in enumerate(timepoints[:-1]):
        n = 0
        while n < n_max:
            # form the function g with g(u) = 0
            for m in range(coll.num_nodes):
                fn[m * nvars: (m + 1) * nvars] = cp.asarray(prob.eval_f(un[m * nvars: (m + 1) * nvars].get(),
                                                             t=timepoint + dt * coll.nodes[m]).flatten())
            g[:] = u0_coll - (un - dt * sp.kron(Q, sp.eye(nvars)).dot(fn))
    
            # if g is close to 0, then we are done
            res_newton = cp.linalg.norm(g, cp.inf)
            #print('Newton:', n, res_newton)
    
            if res_newton < abs_tol: # and res_newton/cp.linalg.norm(u0_coll, cp.inf) < rel_tol:
                break
    
            # assemble dg
            #dg = -sp.eye(nvars * coll.num_nodes) + dt * sp.kron(Q, sp.eye(nvars)).dot(sp.kron(sp.eye(coll.num_nodes), prob.A) + 1.0 / prob.eps**2 * sp.diags((1.0 - (prob.nu + 1) * un ** prob.nu), offsets=0))
            y1 = un[::2]
            y2 = un[1::2]
            diag = cp.empty(len(un))
            upper = cp.zeros(len(un)-1)
            lower = cp.zeros(len(un)-1)
            
            diag[::2] = 1-3 * cp.power(y1, 2) - cp.power(y2, 2)
            diag[1::2] = 3-3 * cp.power(y1, 2) - 9*cp.power(y2, 2)
            upper[::2] = -2*y2-1
            lower[::2] = 2*y1+1
            f_M_prime = sp.diags([upper, diag, lower], offsets=[1, 0, -1])
            dg = -sp.eye(nvars * coll.num_nodes) + dt * sp.kron(QDmat, sp.eye(nvars)) @ f_M_prime

    
            # Newton
            # vk =  sp.linalg.spsolve(dg, g)
            # vk = sp.linalg.gmres(dg, g, x0=cp.zeros_like(vk), maxiter=10, atol=1E-10, callback=count)[0]
            # iter_count += count.niter
            # un -= vk
            # continue
    
            # Collocation
            # Emulate Newton
            # dgP = -sp.eye(nvars * coll.num_nodes) + dt * sp.kron(Q, sp.eye(nvars)).dot(sp.kron(sp.eye(coll.num_nodes), prob.A) + 1.0 / prob.eps**2 * sp.diags((1.0 - (prob.nu + 1) * un ** prob.nu), offsets=0))
            # Linear SDC (e.g. with diagonal preconditioner)
            #dgP = -sp.eye(nvars * coll.num_nodes) + dt * sp.kron(QDmat, sp.eye(nvars)).dot(sp.kron(sp.eye(coll.num_nodes), prob.A) + 1.0 / prob.eps**2 * sp.diags((1.0 - (prob.nu + 1) * un ** prob.nu), offsets=0))
            dgP = -sp.eye(nvars * coll.num_nodes) + dt * sp.kron(QDmat, sp.eye(nvars)) @ f_M_prime
            # Linear SDC with frozen Jacobian
            #dgP = -sp.eye(nvars * coll.num_nodes) + dt * sp.kron(QDmat, sp.csr_matrix(prob.A) + 1.0 / prob.eps**2 * sp.diags((1.0 - (prob.nu + 1) * un[0:nvars] ** prob.nu), offsets=0))
            # dgP = dgP.astype('float64')
    
            # Diagonalization
            # D, V = cp.linalg.eig(Q)
            # Vinv = cp.linalg.inv(V)
            # dg_diag = -sp.eye(nvars * coll.num_nodes) + dt * sp.kron(sp.diags(D), prob.A + 1.0 / prob.eps**2 * sp.diags((1.0 - (prob.nu + 1) * un[0:nvars] ** prob.nu), offsets=0))
            # dg_diag = dg_diag.astype('complex64')
    
            vk = cp.zeros(nvars * coll.num_nodes, dtype='float64') #un.copy() #
            res = cp.zeros(nvars * coll.num_nodes, dtype='float64')
    
            res[:] = g - dg.dot(vk)
            k = 0
            while k < k_max:
                # Newton/Collocation: works well with float32 for both P and res, but not for vk
                # vk += sp.linalg.spsolve(dgP, res)
                vk += sp.linalg.gmres(dgP, res, x0=cp.zeros_like(res), maxiter=100, atol=abs_tol)[0]
    
                # Newton/Diagonalization
                # vk += cp.real(sp.kron(V,sp.eye(nvars)).dot(sp.linalg.spsolve(dg_diag, sp.kron(Vinv,sp.eye(nvars)).dot(res))))
                # vk += cp.real(sp.kron(V,sp.eye(nvars)).dot(sp.linalg.gmres(dg_diag, sp.kron(Vinv,sp.eye(nvars)).dot(res), x0=cp.zeros_like(res), maxiter=1, atol=1E-10, callback=count)[0]))
    
                res[:] = g - dg.dot(vk)
                resnorm = cp.linalg.norm(res, cp.inf)
                rel_err = cp.linalg.norm(vk-un, cp.inf)
                #print('   SDC:', n, k, ksum, resnorm)
    
                k += 1
                ksum += 1
                if resnorm < abs_tol: #and rel_err < rel_tol * cp.linalg.norm(vk, cp.inf):
                    break
    
            # Update
            un -= vk
    
            # increase Newton iteration count
            n += 1
            nsum += 1
        temp = un[-nvars:]
        u_buffer[index+1, :] = temp
        u0_coll = cp.kron(cp.ones(coll.num_nodes), temp).flatten()

    return u_buffer, timepoints, prob, ksum, nsum

In [53]:
res_c, time_c, prob_c, ksum, nsum = nonlinear_auzinger(dt=0.0005, num_nodes=4, abs_tol = 1E-12)
print((ksum, nsum))
cp.linalg.norm(res_c[-1]-cp.asarray(prob_c.u_exact(time_c[-1]).flatten()), cp.inf)

(4714, 4714)


array(6.2857497e-12)